In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
base_url = "https://aarhusbolig.dk"

In [3]:
def get_next_page(html):
    try:
        search_options_bar = html.find("div", class_="search-options-bar l-document")
        nxt = search_options_bar.find("span", "icon-arrow-right prev-next-icon")
        return int(nxt.parent["href"].split("&")[-1].split("=")[1])
    except:
        return None

In [4]:
def get_urls(html):
    container = html.find("div",  {"class": ["housecard-slider w-housecards desktop-only-large"]})
    links = container.find_all("a", class_="housecard-img")
    return [f"{base_url}{a['href']}" for a in links]

In [5]:
def get_page(page):
    url = f"{base_url}/Soeg-bolig?Fritekst=&omr-data-carrier=&BoligOmraader=&org-data-carrier=&Selskaber=&bolig-data-carrier=&BoligTyper=&FloorWish.Min=&FloorWish.Max=&min-data-carrier=&HuslejeMin=&max-data-carrier=&HuslejeMax=&antv-data-carrier=&AntalVaerelser=&antvmax-data-carrier=&AntalVaerelserMax=&div-data-carrier=&BoligArealMin=0&BoligArealMax=200&orderby=&sort=&page={page}"
    result = requests.get(url)
    return BeautifulSoup(result.content, "html.parser")

In [6]:
all_urls = []
next_page = 1
while next_page != None:
    content = get_page(next_page)
    links = get_urls(content)
    print(f"Page {next_page}")
    all_urls += links
    next_page = get_next_page(content)
print(f"DONE! {len(all_urls)} listings")

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
Page 66
Page 67
Page 68
Page 69
Page 70
Page 71
Page 72
Page 73
Page 74
Page 75
Page 76
Page 77
Page 78
Page 79
Page 80
Page 81
Page 82
Page 83
Page 84
Page 85
Page 86
Page 87
Page 88
Page 89
Page 90
Page 91
Page 92
Page 93
Page 94
Page 95
Page 96
Page 97
Page 98
Page 99
Page 100
Page 101
Page 102
Page 103
Page 104
Page 105
Page 106
Page 107
Page 108
Page 109
Page 110
Page 111
Page 112
Page 113
Page 114
Page 115
Page 116
Page 117
Page 118
Page 119
Page 120
Page 121
Page 122
Page 123
P

In [7]:
def get_listing(url):
    result = requests.get(url)
    return BeautifulSoup(result.content, "html.parser")

In [8]:
def to_int(text):
    return int(text.split(" ")[0].split(",")[0].replace(".", ""))

In [9]:
def get_data(listing_url):
    listing_html = get_listing(listing_url)
    details = listing_html.find("div", class_="bolig-details")
    data = {}
    data["url"] = listing_url
    data["address"] = details.find("h1").text
    base_data = details.find("table", class_="afdeling-bolig-data")

    try:
        span = listing_html.find("span", text="Boligorganisation").parent.find("a")["href"]
        data["company"] = re.search(r'www\.(.*)\.', span).groups()[0]
    except:
        print("Failed company", listing_url)
        pass
        
    tds = base_data.find_all("td")
    data["type"] = tds[0].text
    data["rooms"] = to_int(tds[1].text)
    data["listing"] = to_int(tds[2].text)
    data["size_m2"] = to_int(tds[3].text)
    
    for cell in tds[4].find_all("div"):
        spans = cell.find_all("span")
        data[spans[0].text] = to_int(spans[1].text)
    data["price"] = to_int(base_data.find("span", class_="details-last details-sum").text)
    
    try:
        data["lat"], data["long"] = listing_html \
            .find("table", class_="afdeling-bolig-data") \
            .find("span", text="Se på kort") \
            .parent['href'].split("|")[-1].split(",")
        
        # Swapping on purpose to create an unclean dataset
        data["long"], data["lat"] = data["lat"], data["long"]
    except:
        print("Failed location", listing_url)
        pass
    
    data["text"] = listing_html.find("ul", class_="afdeling-details").text.replace("\r", "").replace("\n", "")
    return data

In [10]:
listings = []
total = len(all_urls)
current = 1
for listing_url in all_urls:
    print(f"{current}/{total}")
    listing = get_data(listing_url)
    listings.append(listing)
    current += 1

1/2114
2/2114
3/2114
4/2114
5/2114
6/2114
7/2114
8/2114
9/2114
10/2114
11/2114
12/2114
13/2114
14/2114
15/2114
16/2114
17/2114
18/2114
19/2114
20/2114
21/2114
22/2114
23/2114
24/2114
25/2114
26/2114
27/2114
28/2114
29/2114
30/2114
31/2114
32/2114
33/2114
34/2114
35/2114
36/2114
37/2114
38/2114
39/2114
40/2114
41/2114
42/2114
43/2114
44/2114
45/2114
46/2114
47/2114
48/2114
49/2114
50/2114
51/2114
52/2114
53/2114
54/2114
55/2114
56/2114
57/2114
58/2114
59/2114
60/2114
61/2114
62/2114
63/2114
64/2114
65/2114
66/2114
67/2114
68/2114
69/2114
70/2114
71/2114
72/2114
73/2114
74/2114
75/2114
76/2114
77/2114
78/2114
79/2114
80/2114
81/2114
82/2114
83/2114
84/2114
85/2114
86/2114
87/2114
88/2114
89/2114
90/2114
91/2114
92/2114
93/2114
94/2114
95/2114
96/2114
97/2114
98/2114
99/2114
100/2114
101/2114
102/2114
103/2114
104/2114
105/2114
106/2114
107/2114
108/2114
109/2114
110/2114
111/2114
112/2114
113/2114
114/2114
115/2114
116/2114
117/2114
118/2114
119/2114
120/2114
121/2114
122/2114
123/2114
1

In [12]:
import json
jsonString = json.dumps(listings)
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

# 2. Loading the data

In [13]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)

In [14]:
df = pd.read_json('data.json')

In [15]:
df2 = pd.DataFrame()
df2["listingid"] = df["url"].str.extract(r'bolignr=([0-9]*)')
df2["companyid"] = df["url"].str.extract(r'selskab=([0-9]*)')
df2["company"] = df["company"]
df2["price"] = df["price"]
df2["url"] = df["url"]

df2["long"] = df["long"]
df2["lat"] = df["lat"]

df2["address"] = df["address"]
df2["type"] = df["type"]
df2["rooms"] = df["rooms"]
df2["size_m2"] = df["size_m2"]
df2["listings"] = df["listing"]
df2["text"] = df["text"]

In [16]:
others = [
    'Råderet',
    'Hjemfald',
    'Merleje køkken/bad', 
    'Indv.vedlh',
    'Merinvestering solceller',
    'Merleje flisebelægning',
    'Merleje overdækning','Diverse',
    'Tillæg stor affaldsspand',
    'Etablering Bolignet Århus',
    'Vinduespolering',
    'Etablering af p-pladser',
    'Garage',
    'Afskr. eksisterende køkken',
    'Afskr. eksisterende bad',
    'Afskr. eksisterende skab bad', 
    'Modernisering køkken/bad',
    'Øvrig modernisering', 
    'Råderetsarbejde', 
    'Ydelse indv.mod.', 
    'Vedl. Andel', 
    'Skorstensfejning', 
    'Individuel råderet', 
    'Badmodernisering', 
    'Køkkenmodernisering'
]

colmap = {
    "deposit": ["Indskud:"],
    "rent": ["Boligleje", "Leje", "Nettohusleje", 'Husleje', 'Nettoleje', 'Boligleje, hens.vedl.konto'],
    "electricity": ["Aconto el", 'A conto el', 'ac el', 'A/c el', 'El', 'A/C El', 'A-C el'],
    "heat": ["Aconto varme", 'A conto varme', 'Varme', 'A-C varme', 'A/C Varme', 'ac varme'],
    "water": ["Aconto vand", "A/c vand", 'A conto vand', 'Vand', 'ac vand', 'A/c varme', 'A/C Koldt vand', 'A/C Varmt vand', 'A-C vand', 'Aconto vand/Vandafled.'],
    "tv_internet": ['Antenneafgift', 'Internet, telenet, bredbånd eller bolignet Århus', 'Kabel Tv', 'Antenne/internet', 'Antennebidrag', 'Antenne'],
    "b-ordning": ['B-ordning', 'B-ordn'],
    "other": others
}

In [17]:
for key in colmap:
    df2[key] = df[colmap[key]].sum(axis=1)
df2.replace(0, np.nan, inplace=True)

In [18]:
df2.to_csv("aarhusbolig.csv", index=False)